## Extraktionsbaserad textsammanfattare med olika rankningsmått 

In [1]:
# Imports
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import nltk 
import ssl

# Summarization length of original text
percentage = 0.15

# Fixes some errors, found online at https://github.com/gunthercox/ChatterBot/issues/930#issuecomment-322111087
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [2]:
# Web scrapping -->  module för att ladda ner artiklar 
from newspaper import Article
text = "https://www.aftonbladet.se/nyheter/a/kE6ExL/sd-far-tunga-poster-i-utskotten"#'https://www.svt.se/nyheter/utrikes/stall-dina-fragor-om-kriget-till-svt-s-utrikesreportrar'
article = Article(text, language='sv')
article.download()
article.parse()
#article.nlp()
#article.summary

text = article.text

# Beroende på vilken hemsida nyheten kommer ifrån kan titeln och texten inehålla delar av sidan man egentligen inte bryr sig om
# T.ex. från aftonbladet är titeln med i texten och texten innehåller en mening som: "publicerad: 30 sep", man kan ta bort detta men 
# det blir om vi får tid över.
print('Title:' , article.title, '\n\nText: \n', text)

Title: SD får tunga poster i utskotten 

Text: 
 SD får tunga poster i utskotten

Publicerad: 30 september Uppdaterad: 30 september

Sverigedemokraterna får ordförandeposten i riksdagens justitie- och utrikesutskott.

Nu går ledarna för vänsterblocket till hård attack.

– Det är skrämmande, ganska chockartat, säger Socialdemokraternas gruppledare Lena Hallengren till Aftonbladet.

Sverigedemokraterna , Moderaterna, Kristdemokraterna och Liberalerna har delat upp posterna i utskotten och EU-nämnden.

Där tar Sverigedemokraterna flera viktiga poster.

Bland annat tilldelas partiet ordförandeposten i arbetsmarknadsutskottet, näringsutskottet, justitieutskottet samt utrikesutskottet, enligt ett pressmeddelande.

De erhåller även posten som vice ordförande i civilutskottet, trafikutskottet, försvarsutskottet samt skatteutskottet.

– Det som överraskade mig mest, men som jag kan se varför de vill ha, är ordförandeposten i utrikesutskottet. Det är ett tecken på att de lyckats i förhandlingen 

# Preprocessing 

## Overview
### Calculate number of sentences to keep

### List 1 - sentences
* Varje mening separat

### Dataframe - scores
#### columns are the score of each ranking measure
* Baseline
* Headings
* TF/IDF-score
* NER
* ~~Class~~ //Om vi har tid för ML

### List 2 -> Cleaned for Stop Words 
* Varje mening separat 

###

##### List 1

In [3]:
# removes endlines:
from token import NEWLINE


org_sentences = text.replace('\n\n', '. ')
# creates some exceptions from above rule
org_sentences = org_sentences.replace('.. ', '. ')
org_sentences = org_sentences.replace(':. ', ': ')
org_sentences = org_sentences.split('. ')

org_sentences[0:5]

['SD får tunga poster i utskotten',
 'Publicerad: 30 september Uppdaterad: 30 september',
 'Sverigedemokraterna får ordförandeposten i riksdagens justitie- och utrikesutskott',
 'Nu går ledarna för vänsterblocket till hård attack',
 '– Det är skrämmande, ganska chockartat, säger Socialdemokraternas gruppledare Lena Hallengren till Aftonbladet']

In [4]:
num_of_org_sentences = len(org_sentences)
sentences_to_keep = round(num_of_org_sentences * percentage)
sentences_to_keep

5

##### Dataframe

In [5]:
index = range(org_sentences.__len__())
columns = ['Baseline', 'Headings', 'TF', 'NER']
scores = pd.DataFrame(index=index, columns=columns)
scores.fillna(0, inplace=True)
scores.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Baseline  34 non-null     int64
 1   Headings  34 non-null     int64
 2   TF        34 non-null     int64
 3   NER       34 non-null     int64
dtypes: int64(4)
memory usage: 1.2 KB


### Stop Word Filtering
* Hur stor korpus ska vi ha? 

In [6]:
# https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
#nltk.download('stopwords')

swe_stop_words = set(stopwords.words('swedish'))
#print(swe_stop_words)
swe_stop_words.update([',', '"', ':', '-', '–', '”'])

word_tokens = word_tokenize(' '.join(org_sentences))
  
filtered_sentence = [w for w in word_tokens if not w.lower() in swe_stop_words]

fdist = FreqDist(word.lower() for word in word_tokenize(' '.join(filtered_sentence)))

#print(filtered_sentence)
#print(fdist.most_common())
print(fdist.items())

dict_items([('sd', 2), ('får', 5), ('tunga', 1), ('poster', 2), ('utskotten', 2), ('publicerad', 1), ('30', 2), ('september', 2), ('uppdaterad', 1), ('sverigedemokraterna', 4), ('ordförandeposten', 4), ('riksdagens', 1), ('justitie-', 1), ('utrikesutskott', 1), ('går', 2), ('ledarna', 1), ('vänsterblocket', 2), ('hård', 1), ('attack', 2), ('skrämmande', 2), ('ganska', 2), ('chockartat', 2), ('säger', 5), ('socialdemokraternas', 1), ('gruppledare', 2), ('lena', 2), ('hallengren', 2), ('aftonbladet', 2), ('moderaterna', 2), ('kristdemokraterna', 1), ('liberalerna', 1), ('delat', 1), ('posterna', 1), ('eu-nämnden', 1), ('tar', 1), ('flera', 2), ('viktiga', 1), ('bland', 1), ('annat', 1), ('tilldelas', 1), ('partiet', 1), ('arbetsmarknadsutskottet', 1), ('näringsutskottet', 1), ('justitieutskottet', 1), ('samt', 2), ('utrikesutskottet', 7), ('enligt', 1), ('pressmeddelande', 1), ('erhåller', 1), ('även', 2), ('posten', 1), ('vice', 1), ('ordförande', 2), ('civilutskottet', 1), ('trafikutsk

In [7]:
# Perform Stop Word Filtering On Example (Original Text) 
def stop_word_filtering(original_text): 
    swf_text = original_text.removeall 

    return swf_text

### Lemmatizer & NER 

* LEmmatizer verkar ok 
* NER --> helt klart bristande -- men det kan duga? 

In [19]:
from html import entities
import spacy
# Credit to Explosion for sv_core_news_sm --> https://github.com/explosion 
nlp = spacy.load("sv_core_news_sm")
doc = nlp(' '.join(org_sentences))

named_entities = doc.ents.__str__()
print(type(named_entities))

lemmatized_org_sentences = []
for i, sentence in enumerate(filtered_sentence):
    for doc_sentence in nlp(sentence): 
        for word in doc_sentence: 
            lemmatized_org_sentences[i].add(word.lemma_)
    pass



<class 'str'>


TypeError: 'spacy.tokens.token.Token' object is not iterable

# Ranking Measures

### Baseline
(1/N --> n=ordning mening kommer i dvs. första meningen får N=1 -> 1/1, andra meningen får N=2 -> 1/2, osv.)

In [10]:
# Ranking metric 1 --> Baseline
for i, score in enumerate(scores['Baseline']) :
    scores['Baseline'][i] = 1/((i+1))
scores.describe()

,Baseline,Headings,TF,NER
count,34.000000,34.0,34.0,34.0
mean,0.121124,0.0,0.0,0.0
std,0.183991,0.0,0.0,0.0
min,0.029412,0.0,0.0,0.0
25%,0.038846,0.0,0.0,0.0
50%,0.057190,0.0,0.0,0.0
75%,0.108333,0.0,0.0,0.0
max,1.000000,0.0,0.0,0.0


### Headings

In [11]:
# Ranking metric 2 --> Headings
# Sets all 'Headings' scores to 0, mostly for testing so i can run this multiple times, 
# but also to make sure nothing weird has happened earlier in the code.
scores['Headings'] = 0
for i, sentence in enumerate(org_sentences):
    for word in article.title.split(' '):
        if word in sentence:
            scores.at[i, 'Headings'] += 1
scores.describe()

,Baseline,Headings,TF,NER
count,34.000000,34.000000,34.0,34.0
mean,0.121124,1.352941,0.0,0.0
std,0.183991,1.011520,0.0,0.0
min,0.029412,0.000000,0.0,0.0
25%,0.038846,1.000000,0.0,0.0
50%,0.057190,1.000000,0.0,0.0
75%,0.108333,1.000000,0.0,0.0
max,1.000000,6.000000,0.0,0.0


## TERM FREQUENCY 


In [12]:
#
scores['TF'] = 0

for i, sentence in enumerate(org_sentences):
    for word in sentence.split(' '):
        word = word.lemma
        if word in fdist.keys():
            scores.at[i, "TF"] += fdist.freq(word)

scores.describe()

,Baseline,Headings,TF,NER
count,34.000000,34.000000,34.000000,34.0
mean,0.121124,1.352941,0.032110,0.0
std,0.183991,1.011520,0.023893,0.0
min,0.029412,0.000000,0.000000,0.0
25%,0.038846,1.000000,0.018349,0.0
50%,0.057190,1.000000,0.027523,0.0
75%,0.108333,1.000000,0.042813,0.0
max,1.000000,6.000000,0.125382,0.0


### TF*IDF-score
* Diskutera hur vi kan använda måtten 
* If similarity is close --> Similar content --> Remove redundance?  



https://forketyfork.medium.com/latex-math-formulas-a-cheat-sheet-21e5eca70aae

In [13]:
# Ranking metric 3 --> TF*IDF

# Term Weights --> Calculate importance of single words in text/doc
# Binary term weights --> document specific
# TF*IDF term weights --> document-collection specific 

# Assign weights to each dimension (attr/word) of each sentence (record/example) 

# Term Frequency (TF-score) --> TFij == frequency of the jth term in in the ith doc 

# Inverse Document Frequency 
# idf-score of the jth term measures the uniqueness of the jth term in the collection of documents
# IDFj = log(M / Nj)
#
# M = total num of docs in collection 
# Nj is the number of documents that contain the jth term

# HIGH TF*IDF-score 
# Word frequent in document && Occur in few documents of the collection 
# LOW TF*IDF-score
# Not present in document || present in all documents of the collection 

### NER 
* (nltk lib) --> (Meningar med Named Entities är troligtvis viktigare)

In [14]:
scores['NER'] = 0

# Hittar inte allt p.g.a sentece.split(' ') där den splittar namn som t.ex "ulf kristensson" till "ulf" och "kristensson"
for i, sentence in enumerate(org_sentences) :
    for word in sentence.split(' '):
        if word in named_entities.split(' '):
            print(i, word)
            scores.at[i, "NER"] += 1

scores.describe()


4 Lena
5 Moderaterna,
10 KD,
10 Aftonbladets
10 My
16 Ulf
16 Lena
16 Hallengren,
25 Martin
25 Markus
26 Marcus
26 Martin
26 Kinnunen,
31 Jakob


,Baseline,Headings,TF,NER
count,34.000000,34.000000,34.000000,34.000000
mean,0.121124,1.352941,0.032110,0.411765
std,0.183991,1.011520,0.023893,0.924995
min,0.029412,0.000000,0.000000,0.000000
25%,0.038846,1.000000,0.018349,0.000000
50%,0.057190,1.000000,0.027523,0.000000
75%,0.108333,1.000000,0.042813,0.000000
max,1.000000,6.000000,0.125382,3.000000


# Combination Function

## Overview
### Standardize
* standardize all scores
### Combine
* Combine the scores into one overall score
* add weight and/or ML if time allows

In [15]:
# Standardize
scores_standardized = StandardScaler.fit_transform(self=StandardScaler(), X=scores)
scores_standardized = pd.DataFrame(scores_standardized, columns=columns)
scores_standardized

,Baseline,Headings,TF,NER
0,4.848572,4.663223,-0.064957,-0.451848
1,2.090179,-0.354169,-0.324785,-0.451848
2,1.170715,0.649309,0.194871,-0.451848
3,0.710982,-0.354169,-0.324785,-0.451848
4,0.435143,-0.354169,-0.194871,0.645497
5,0.251250,1.652788,-0.844441,0.645497
6,0.119898,0.649309,-0.584613,-0.451848
7,0.021384,-0.354169,0.194871,-0.451848
8,-0.055238,-0.354169,0.194871,-0.451848
9,-0.116536,-0.354169,0.454699,-0.451848


In [16]:
# Combination Function
# Här ligger ML om vi gör det  

final_score = scores_standardized.sum(axis=1)
best_sentences = final_score.nlargest(sentences_to_keep, keep='all').index.values
print(best_sentences)


[ 0 32 16 26 10]


# Assemble output 
* Reassemble according to overall score ranking
* Output summarization 

In [17]:
# Assemble Output 
print("Percentage:\n")
for i in best_sentences: 
    print(org_sentences[i])

print("\n\n")

print("N sentences:\n")
for i in best_sentences[0:3]: 
    print(org_sentences[i])

Percentage:

SD får tunga poster i utskotten
– Det här är en oerhört viktig post som kanske är mer representativ än andra utskottsposter, men man måste komma ihåg att inom utrikespolitiken har vi en regering, en utrikesminister och en försvarsminister som har mycket mer verkställande makt, säger han och tillägger: – Frågan är hur stort inflytande ordförandefrågorna får om riksdagspartierna förhandlar fram överenskommelser i förväg
Det är tyvärr ett tecken på vart vi är på väg, att för Ulf Kristerssons del är det viktigt att få makt och inflytande, man säljer sig billigt, säger Lena Hallengren, gruppledare för Socialdemokraterna och fortsätter: – Utrikesutskottet är otroligt betydelsefullt med en omvärld som är minst sagt orolig
”För utrikesutskottet antar jag att det står mellan Marcus Wiechel och Martin Kinnunen, här bägge på delegationsresa på eget initiativ och mot UD:s avrådan, till Syrien 2017”, skriver han på Twitter
Det är ett tecken på att de lyckats i förhandlingen med M och K

In [18]:
# Newspaper Summarization
article.nlp()
print("\nNewspaper3k: \n", article.summary)


Newspaper3k: 
 SD får tunga poster i utskottenPublicerad: 30 september Uppdaterad: 30 septemberSverigedemokraterna får ordförandeposten i riksdagens justitie- och utrikesutskott.
Utrikesutskottet är inte vilket som helst, det ska representera Sveriges riksdag utomlands och överhuvudtaget i relationen till omvärlden, så det är en viktig position.
Den här gången genom att ge Sverigedemokraterna ordförandeposten i utrikesutskottet”, skriver hon på Twitter.
I detta känsliga säkerhetspolitiska läge, när Sverige ska bli medlem i Nato och vara ordförande i EU.
”Får mindre inflytande”Utrikespolitiska institutets direktör Jakob Hallgren tror dock inte att utskottet kommer att ha någon större inverkan på den svenska utrikespolitiken.
